<a href="https://colab.research.google.com/github/ArmstrongVo/VoTrongLuan.github.io/blob/main/TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import libraries
import keras
import cv2
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# access to data
test='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TEST'
train='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TRAIN'

In [7]:
# để hệ thống đạt được độ chính xác cao nhất
# tiến hành pre-processing sử dụng kỹ thuật data agumentation
def equalize_histogram(image):   
    # chuẩn hóa độ sáng trong khoảng [0-1] : tối ưu độ sáng và chi tiết trong ảnh
    image = image.astype(np.float32) / 255.0
   
    # kiểm tra nếu hình ảnh có nhiều hơn 2 chiều thì chuyển đổi sang ảnh xám
    if len(image.shape) > 2:  
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Cân bằng histogram 
    equalized_image = cv2.equalizeHist(image)
    
    return equalized_image

# thiết lập các thông số cho data augmentation
# cân bằng histogram
# xoay ảnh
# lật chiều ngang
# lật dọc
# phóng to hoặc thu nhỏ
train_augen = ImageDataGenerator(preprocessing_function=equalize_histogram,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)

test_augen = ImageDataGenerator(  preprocessing_function=equalize_histogram,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)
# data augumentation
train_set = train_augen.flow_from_directory(train,
                                                 target_size=(64, 64),
                                                 batch_size=10,
                                                 class_mode='categorical')
test_set = test_augen.flow_from_directory(test,
                                            target_size=(64, 64),
                                            batch_size=10,
                                            class_mode='categorical')




Found 5478 images belonging to 3 classes.
Found 370 images belonging to 3 classes.
